# Import Data

## Présentation
Lorsqu'il est nécessaire de mettre à jour la base de données MongoDB avec de nouvelles données, il peut être pratique de supprimer tous les documents (Tweets) de la collection existante et d'insérer les nouveaux documents à partir des fichiers JSON contenu dans le dossier ``data`` à la racine du projet pour garantir la cohérence des données et s'assurer que nous ne disposons pas de données modifiées. Dans ce cas-ci, ce script peut être utilisé.

Que fait-il ?
1. Suppression des données existantes dans la base de données MongoDB ayant pour nom Tweet
2. Lecture des données présente dans le dossier ``data`` à la racine du projet. A noter que nous n'avons pas commit ces données à cause d'une taille volumineuse. Elles peuvent être téléchargées via le lien suivant : [Cliquer ici](https://www.dropbox.com/s/qfhaobip55xxkif/Tweet%20Worldcup.zip?dl=0)
3. Insertions des données lues dans la base de données

## Le code
On importe les différentes bibliothèques nécessaires.

In [53]:
import pymongo
import json
import os
from datetime import datetime
import numpy as np

Connexion à la base de données MongoDB

In [54]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["Tweet"]
collection = db["dataset"]

Suppression de tous les documents de la collection

In [55]:
collection.delete_many({})

In [ ]:
tweet_features_to_keep = ["created_date","id","text","entities","favorite_count","lang","possibly_sensitive","retweet_count","source","user"]
user_features_to_keep = ["friends_count","favourites_count","created_at","id","verified","followers_count","statuses_count"]
def filter_features(tweet):
    tweet_cp = tweet.copy()
    for key in tweet.keys():
        if key not in tweet_features_to_keep:
            del tweet_cp[key]
    for key in tweet["user"].keys():
        if key not in user_features_to_keep:
            del tweet_cp["user"][key]
    return tweet_cp

Définition de la fonction pour upload dans mongodb tout les documents contenus dans un fichier

In [57]:
def upload_file_data(path):
    with open(path, encoding='UTF-8') as f:
        docs = []
        for line in f:
            doc = json.loads(line)
            doc = filter_features(doc)
            doc["created_date"] = datetime.strptime(doc["created_date"],"%Y-%m-%d %H:%M:%S")
            doc["user"]["created_at"] = datetime.strptime(doc["user"]["created_at"], "%a %b %d %H:%M:%S %z %Y")
            docs.append(doc)
        if len(docs) > 0:
            collection.insert_many(docs)


Récupération des noms des fichiers JSON dans le dossier approprié

In [58]:
path = "../../data"
all_files = os.listdir(path)

On parcourt l'ensemble des fichiers pour upload les documents qui le constitue tout en affichant des statistiques pour suivre l'évolution de l'importation des données dans MongoDB

In [59]:
start_time = datetime.now()
print(f">>>> Start : {start_time}")

# begins upload
nb_files = len(all_files)
for idx_files, file in enumerate(all_files):
    start_file_time = datetime.now() 
    if file.endswith(".json"):
        upload_file_data(f"{path}/{file}")
    time_for_this_file = (datetime.now()  - start_file_time).seconds
    percentage = round(idx_files*100/nb_files,2)
    print(f" - Upload files n°{idx_files}/{nb_files} in {time_for_this_file}s ({percentage}%)")

# some stats
end_time = datetime.now()     
print(f">>>> End : {end_time}")
diff_time = end_time - start_time
print(f"Upload done in {diff_time.seconds // 3600}h {(diff_time.seconds // 60) % 60}min {diff_time.seconds % 60}sec ")

>>>> Start : 2023-04-28 15:17:09.937574
 - Upload files n°0/2285 in 0s (0.0%)
 - Upload files n°1/2285 in 0s (0.04%)
 - Upload files n°2/2285 in 0s (0.09%)
 - Upload files n°3/2285 in 0s (0.13%)
 - Upload files n°4/2285 in 0s (0.18%)
 - Upload files n°5/2285 in 0s (0.22%)
 - Upload files n°6/2285 in 0s (0.26%)
 - Upload files n°7/2285 in 0s (0.31%)
 - Upload files n°8/2285 in 0s (0.35%)
 - Upload files n°9/2285 in 0s (0.39%)
 - Upload files n°10/2285 in 0s (0.44%)
 - Upload files n°11/2285 in 0s (0.48%)
 - Upload files n°12/2285 in 0s (0.53%)
 - Upload files n°13/2285 in 0s (0.57%)
 - Upload files n°14/2285 in 0s (0.61%)
 - Upload files n°15/2285 in 0s (0.66%)
 - Upload files n°16/2285 in 0s (0.7%)
 - Upload files n°17/2285 in 0s (0.74%)
 - Upload files n°18/2285 in 0s (0.79%)
 - Upload files n°19/2285 in 0s (0.83%)
 - Upload files n°20/2285 in 0s (0.88%)
 - Upload files n°21/2285 in 0s (0.92%)
 - Upload files n°22/2285 in 0s (0.96%)
 - Upload files n°23/2285 in 0s (1.01%)
 - Upload fi

In [60]:
collection.create_index([("id",1)])
collection.create_index([("user.id",1)])
collection.create_index([("user.id",1),("id",1)])
collection.create_index([("user.id",1),("created_date",1)])

'user.id_1_created_date_1'

In [65]:
pipeline = [
    {"$group": {"_id": "$id", "count": {"$sum": 1}, "dups": {"$addToSet": "$_id"}}},
    {"$match": {"count": {"$gt": 1}}}
]

duplicates = []
for doc in collection.aggregate(pipeline):
    duplicates.extend(doc["dups"][1:])

collection.delete_many({"_id": {"$in": duplicates}})